In [13]:
class User:
    def __init__(self, user_id, url, member_since):
        self.user_id = user_id
        self.url = url
        self.member_since = member_since

        # Rest are not initialized here, but in the respective classes
        self.stats = None
        self.about = None
        self.collective = None
        self.communities = None
        self.badges = None
    
    def get_stats(self):
        self.stats = Stats._get_stats(self.user_id)
        return self.stats
    
    def get_about(self, user_id):
        self.about = About(user_id)

    def get_collective(self, user_id):
        self.collective = Collective(user_id)

    def get_communities(self, user_id):
        self.communities = Communities(user_id)

    def get_badges(self, user_id):
        self.badges = Badges(user_id)
 
    def __repr__(self):
        return f"User(id={self.user_id}, url={self.url}, member_since={self.member_since})"


In [14]:
import requests
from bs4 import BeautifulSoup as Bea

class Stats:
    def __init__(self, username, reputation, reached, answers, questions):
        self.username = username
        self.reputation = reputation
        self.reached = reached
        self.answers = answers
        self.questions = questions

    @staticmethod
    def _get_stats(user_id):
        """
        Fetch the HTML using user_id, parse it, and extract all stats to create a Stats instance.
        """
        # Fetch and parse HTML
        soup = Stats._fetch_html(user_id)

        # Extract attributes
        username = Stats.get_username(soup)
        reputation = Stats.get_reputation(soup)
        reached = Stats.get_reached(soup)
        answers = Stats.get_answers(soup)
        questions = Stats.get_questions(soup)

        return Stats(username, reputation, reached, answers, questions)

    @staticmethod
    def _fetch_html(user_id):
        """
        Fetch the HTML for a user's profile page using their user_id.
        """
        url = f"https://stackoverflow.com/users/{user_id}"
        response = requests.get(url)
        response.raise_for_status()  # Raise an error if the request fails
        return Bea(response.content, "html.parser")

    @staticmethod
    def get_username(soup):

        title_element = soup.find("title")
        if title_element:
            title_text = title_element.text.strip()
            # The title typically has the format: "User {username} - Stack Overflow"
            if "User " in title_text:
                return title_text.split("User ")[1].split(" - Stack Overflow")[0]
        return None

    @staticmethod
    def get_reputation(soup):

        container = soup.find(id="stats")
        if container:
            reputation = container.find_all(class_="fs-body3")[0].text.strip()
            return int(reputation.replace(",", ""))  # Convert to integer
        return 0
    
    @staticmethod
    def get_reached(soup):

        container = soup.find(id="stats")
        if container:
            reputation = container.find_all(class_="fs-body3")[1].text.strip()
            return str(reputation.replace(",", ""))  # Convert to integer
        return 0

    @staticmethod
    def get_answers(soup):

        container = soup.find(id="stats")
        if container:
            reputation = container.find_all(class_="fs-body3")[2].text.strip()
            return str(reputation.replace(",", ""))  # Convert to integer
        return 0

    @staticmethod
    def get_questions(soup):
        """
        Extract the number of answers from the HTML.
        """
        container = soup.find(id="stats")
        if container:
            reputation = container.find_all(class_="fs-body3")[3].text.strip()
            return str(reputation.replace(",", ""))  # Convert to integer
        return 0

    def __repr__(self):
        return (f"Stats(username={self.username}, reputation={self.reputation}, "
                f"answers={self.answers}, questions={self.questions})")


In [22]:
user = User(user_id=6309, url="https://stackoverflow.com/users/12345", member_since="2020-01-01")

# Fetch and display stats
user_stats = Stats._get_stats(6309)  
user.stats = user_stats

print(user.stats)

Stats(username=VonC, reputation=1310915, answers=29734, questions=16)
